In [17]:
import pandas as pd
import numpy as np
import os

In [20]:
%%time 

prediction_data = pd.read_csv(f"/kaggle/input/playground-series-s4e8/sample_submission.csv", 
                     index_col = ["id"]).drop("class", axis=1)

CPU times: user 397 ms, sys: 5.89 ms, total: 403 ms
Wall time: 402 ms


<h2>External Results</h2>

1. https://www.kaggle.com/code/arunklenin/ps4e8-binary-class-mathews-correlation-coeff
2. https://www.kaggle.com/code/mobinapoulaei/autogloun-t8-dslanders
3. https://www.kaggle.com/code/olegpushs/automl-grand-prix-8th-place-notebook
4. https://www.kaggle.com/code/samiraalipour/automl-poisonous-mushrooms-t6-dslanders
5. https://www.kaggle.com/code/adaluodaa/mario-s-nightmare-denselight-0-990

<h2>Internal Results</h2>

1. Autogluon with stack level = 2(15 hours)
2. XGBoost with manually generated hyperparameters + One-hot Encoding + SHAP + IsolationForeset(16min)


In [23]:
%%time 
#External Results

prediction_data["sub1"] = pd.read_csv(f"/kaggle/input/ps4e8-binary-class-mathews-correlation-coeff/submission.csv").iloc[:,-1].values.flatten()
# 98521 - 3
prediction_data["sub2"] = pd.read_csv(f"/kaggle/input/autogloun-t8-dslanders/submission.csv").iloc[:,-1].values.flatten()
# 98530 - 1
prediction_data["sub3"] = pd.read_csv(f"/kaggle/input/ex1-pss4e8-autogluon/[AutoML Grand Prix] Team GD Autogluon,mcc.csv").iloc[:,-1].values.flatten()
# 98517 - 5
prediction_data["sub4"] = pd.read_csv(f"/kaggle/input/automl-poisonous-mushrooms-t6-dslanders/submission.csv").iloc[:,-1].values.flatten()
# 98520 - 4
prediction_data["sub5"] = pd.read_csv(f"/kaggle/input/mario-s-nightmare-15-th-place-solution/submission.csv").iloc[:,-1].values.flatten()
# 98528 - 2

print()
display(prediction_data.head(10).style.set_caption(f"Submission file with components"))

,sub1,sub2,sub3,sub4,sub5
id,,,,,
3116945,e,e,e,e,e
3116946,p,p,p,p,p
3116947,p,p,p,p,p
3116948,p,p,p,p,p
3116949,e,e,e,e,e
3116950,e,e,e,e,e
3116951,e,e,e,e,e
3116952,p,p,p,p,p
3116953,p,p,p,p,p


CPU times: user 1.99 s, sys: 24 ms, total: 2.01 s
Wall time: 2.02 s


In [57]:
#Internal Results: XGBoost

import pandas as pd
import numpy as np  
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import IsolationForest

import shap

rs = 101


In [59]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

df_train.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [61]:
df_train_cleaned = df_train.copy()
df_test_cleaned = df_test.copy()

# Drop 'id' column
df_train_cleaned = df_train_cleaned.drop(['id'], axis=1)

# Define the target column
target_column = 'class'

# ordinal column
ordinal_columns = np.array(['gill-spacing'])
# ordinal column에서 사용된 데이터의 순서를 정의
gill_spacing_order = [[ 'f', 'Unknown', 'c', 'd']]

# Select categorical columns, excluding the target column
categorical_columns = df_train_cleaned.select_dtypes(include=['object']).columns.drop(target_column)
categorical_columns = categorical_columns.drop('gill-spacing')

# Select numerical columns, excluding the target column if it's numerical
numerical_columns = df_train_cleaned.select_dtypes(exclude=['object']).columns.drop(target_column, errors='ignore')

In [63]:
# Define a function to identify and replace infrequent categories
def replace_infrequent_categories(df, column, threshold=70):
    value_counts = df[column].value_counts()
    infrequent = value_counts[value_counts <= threshold].index
    df[column] = df[column].apply(lambda x: "Unknown" if x in infrequent else x)
    return df

# Handle invalid values and infrequent categories for all categorical columns
for col in categorical_columns:
    df_train_cleaned = replace_infrequent_categories(df_train_cleaned, col)
    df_test_cleaned = replace_infrequent_categories(df_test_cleaned, col)

# ordinal column에도 똑같이 적용
df_train_cleaned = replace_infrequent_categories(df_train_cleaned, ordinal_columns[0])
df_test_cleaned = replace_infrequent_categories(df_test_cleaned, ordinal_columns[0])

In [64]:
# numercal column들의 skewness (데이터의 전체적인 기울기) 계산
df_train_cleaned[numerical_columns].apply(lambda x: stats.skew(x.dropna()))


cap-diameter    3.972607
stem-height     1.926681
stem-width      1.235426
dtype: float64

In [65]:
# Compute medians for numerical columns in the training set
medians = df_train_cleaned[numerical_columns].median()

# Fill missing values in the training and testing sets
df_train_cleaned[numerical_columns] = df_train_cleaned[numerical_columns].fillna(medians)
df_test_cleaned[numerical_columns] = df_test_cleaned[numerical_columns].fillna(medians)

In [66]:
# Impute any missing values with 'Unknown'
df_train_cleaned = df_train_cleaned.fillna("Unknown")
df_test_cleaned = df_test_cleaned.fillna("Unknown")

In [67]:
#ㅁㄴㅇ
df_train_cleaned = df_train_cleaned.drop_duplicates()

In [68]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the target variable
train_encoded_target = label_encoder.fit_transform(df_train_cleaned[['class']])

# Convert categorical columns to 'category' dtype 
df_train_cleaned[categorical_columns] = df_train_cleaned[categorical_columns].astype('category')
df_test_cleaned[categorical_columns] = df_test_cleaned[categorical_columns].astype('category')

# Convert ordinal columns to 'category' dtype 
df_train_cleaned[ordinal_columns] = df_train_cleaned[ordinal_columns].astype('category')
df_test_cleaned[ordinal_columns] = df_test_cleaned[ordinal_columns].astype('category')

# Define the numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('convert_to_float32', FunctionTransformer(lambda x: x.astype(np.float32)))
])

# Define the categorical pipeline
ordinal_pipeline = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(dtype=np.int32, handle_unknown='use_encoded_value', unknown_value=-1, categories=gill_spacing_order))
])

# Define the categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', dtype=np.int32, handle_unknown='ignore'))
])

# Combine both numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_columns),
        ('ord', ordinal_pipeline, ordinal_columns),
        ('cat', categorical_pipeline, categorical_columns)
    ]
)


# Apply the transformations using the pipeline
df_train_encoded = preprocessor.fit_transform(df_train_cleaned)
df_test_encoded = preprocessor.transform(df_test_cleaned)

# Ensure outputs are dense arrays
train_encoded_dense = df_train_encoded.toarray()
test_encoded_dense = df_test_encoded.toarray()

# Get feature names
numerical_feature_names = numerical_columns  # Assuming numerical columns do not change names
ordinal_feature_names = ordinal_columns
categorical_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_columns)

# Combine the feature names
all_feature_names = np.concatenate([numerical_feature_names, ordinal_feature_names, categorical_feature_names])

# Debugging: Print the number of feature names
print("Number of Features:", len(all_feature_names))

# Convert the transformed dense arrays back into DataFrames
df_train_preprocessed = pd.DataFrame(train_encoded_dense, columns=all_feature_names)
df_test_preprocessed = pd.DataFrame(test_encoded_dense, columns=all_feature_names)

Number of Features: 120


In [69]:
# Apply Isolation Forest for outlier detection
isolation_forest = IsolationForest(contamination=0.05, random_state=42)
outlier_labels = isolation_forest.fit_predict(df_train_preprocessed)

# Filter out outliers
non_outliers_mask = outlier_labels != -1
df_train_preprocessed = df_train_preprocessed[non_outliers_mask]
train_encoded_target = train_encoded_target[non_outliers_mask]

In [70]:

high_negative_shap = ['cap-surface_l', 'gill-color_e', 'cap-color_b', 'gill-color_f', 'veil-color_y', 'stem-color_b', 'ring-type_m', 'stem-color_l']
# 보류: 'cap-color_o', 'habitat_g', 'cap-surface_y' not working as expected
# 'gill-attachment_f', 'habitat_l'

df_train_preprocessed = df_train_preprocessed.drop(high_negative_shap, axis = 1)
df_test_preprocessed = df_test_preprocessed.drop(high_negative_shap, axis = 1)

In [71]:
# Separate features (X) and target variable (y)
X = df_train_preprocessed
y = train_encoded_target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)

In [72]:
#duplicate
params = {
    'colsample_bytree': 0.4, 
    'learning_rate': 0.01, 
    'max_depth': 14, 
    'min_child_weight': 1, 
    'n_estimators': 3000, 
    'subsample': 0.9,
    'use_label_encoder': False,  
    'eval_metric': 'mlogloss',   
    'session_id': 112,   
    'device': 'cuda'
    # 0.9846626117489504
}

# Initialize the XGBClassifier with the defined parameters
xgb_model = XGBClassifier(**params)

# Fit the model to the training data 
xgb_model.fit(X_train, y_train)

# Predict on the test data 
y_pred = xgb_model.predict(X_test)

# Evaluate the model using Matthews correlation coefficient
mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient:", mcc)

Matthews Correlation Coefficient: 0.9842937358629338


In [73]:
# Fit the model to the training data 
xgb_model.fit(X_test, y_test)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=14,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=3000,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [74]:
print("original data target distribution")
print(df_train['class'].value_counts())
print(df_train['class'].value_counts() / len(df_train))

original data target distribution
class
p    1705396
e    1411549
Name: count, dtype: int64
class
p    0.547137
e    0.452863
Name: count, dtype: float64


In [75]:
result = pd.DataFrame(y_pred)
print("train result target label distribution")
print(result[0].value_counts())
print(result[0].value_counts() / len(y_pred))

train result target label distribution
0
1    317736
0    274485
Name: count, dtype: int64
0
1    0.536516
0    0.463484
Name: count, dtype: float64


In [76]:
test_preds = xgb_model.predict(df_test_preprocessed)
test_preds = label_encoder.inverse_transform(test_preds)

In [77]:
result = pd.DataFrame(df_test['id'])
result['class'] = test_preds
print("test result target distribution")
print(result['class'].value_counts())
print(result['class'].value_counts() / len(test_preds))

test result target distribution
class
p    1133284
e     944680
Name: count, dtype: int64
class
p    0.545382
e    0.454618
Name: count, dtype: float64


In [78]:
prediction_data['internal_sub1'] = pd.DataFrame({'id': df_test['id'],
                       'class': test_preds}).iloc[:,-1].values.flatten()

prediction_data['internal_sub1'].head()

id
3116945    e
3116946    p
3116947    p
3116948    p
3116949    e
Name: internal_sub1, dtype: object

In [79]:
# Autogluon with stack level=2, runtime= 15hours
"""
Due to memory error, this could not be run on Kaggle notebook environment. The result was generated
from the local environment

import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer, LabelEncoder, OneHotEncoder
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import IsolationForest

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train = df_train.drop(['id'], axis = 1)
df_test = df_test.drop(['id'], axis = 1)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the target variable
df_train['class'] = label_encoder.fit_transform(df_train[['class']])

# Install AutoGluon
!conda install ray==2.10.0 --yes
!conda install autogluon.tabular --yes
!conda install ipywidgets --yes

from autogluon.tabular import TabularPredictor
# from autogluon.features.generators import FillNaFeatureGenerator

## Map target values: 'e' -> 1, 'p' -> 0
label = 'class'
# train_data[label] = train_data[label].map({'e': 1, 'p': 0})

# Train model with AutoGluon
predictor = TabularPredictor(
    label=label,
    eval_metric='mcc',
    problem_type='binary'
).fit(
    df_train,
    presets='best_quality',
    #feature_generator=feature_generator,  # Use the custom feature generator
    time_limit=3600*15,
    num_bag_folds=5,
    verbosity=3,
    excluded_model_types=['KNN'],
    num_cpus = 'auto',
    num_gpus = 'auto',
    num_stack_levels = 2,  # enables the stacking of models
    #hyperparameter_tune_kwargs='auto',
)

'''
num_stack_level
Number of stacking levels in stack ensemble. 
Increases training time by approximately num_stack_levels+1. 
Default is 0 (disabled). Recommended: 1-3 levels for better predictive performance. 
Ensure num_bag_folds ≥ 2 to avoid overfitting, or a ValueError will occur.
'''

# Print fit summary
results = predictor.fit_summary()
print(results)

df_test = pd.read_csv('test.csv')

# Create a submission DataFrame
submission = pd.DataFrame({
    'id': df_test['id'],
    'class': y_pred
})

# Save the predictions to a CSV file
submission.to_csv('submission.csv', index=False)

"""




'\nDue to memory error, this could not be run on Kaggle notebook environment. The result was generated\nfrom the local environment\n\nimport pandas as pd\nimport numpy as np  \nimport matplotlib.pyplot as plt\nfrom scipy import stats\nimport seaborn as sns\nimport re\n\nimport warnings\nwarnings.filterwarnings("ignore")\n\nimport sklearn\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer, LabelEncoder, OneHotEncoder\nfrom sklearn.metrics import matthews_corrcoef\nfrom sklearn.ensemble import IsolationForest\n\ndf_train = pd.read_csv(\'train.csv\')\ndf_test = pd.read_csv(\'test.csv\')\n\ndf_train = df_train.drop([\'id\'], axis = 1)\ndf_test = df_test.drop([\'id\'], axis = 1)\n\n# Initialize LabelEncoder\nlabel_encoder = LabelEncoder()\n\n# Fit and transform the target variable\ndf_train[\'class\'] = label_encoder.fit_transform(df_train[[\'class\']])\n\n# Install Auto

In [80]:
prediction_data['internal_sub2'] = pd.read_csv(f"/kaggle/input/autogluon-2stacklevel/submission.csv").iloc[:,-1].values.flatten()
prediction_data['internal_sub2'] = prediction_data['internal_sub2'].apply(lambda x: 'e' if x == 0 else 'p')
prediction_data['internal_sub2'].head()

id
3116945    e
3116946    p
3116947    p
3116948    p
3116949    e
Name: internal_sub2, dtype: object

In [81]:
prediction_data

,sub1,sub2,sub3,sub4,sub5,internal_sub1,internal_sub2
id,,,,,,,
3116945,e,e,e,e,e,e,e
3116946,p,p,p,p,p,p,p
3116947,p,p,p,p,p,p,p
3116948,p,p,p,p,p,p,p
3116949,e,e,e,e,e,e,e
...,...,...,...,...,...,...,...
5194904,p,p,p,p,p,p,p
5194905,p,p,p,p,p,p,p
5194906,p,p,p,p,p,p,p


In [82]:

for i in range(1, len(prediction_data.columns) - 1):
    prediction_data[f"sub{i}"] = prediction_data[f"sub{i}"].apply(lambda x: 0 if x == 'e' else 1)

for i in range(1, 3):
    prediction_data[f"internal_sub{i}"] = prediction_data[f"internal_sub{i}"].apply(lambda x: 0 if x == 'e' else 1)


In [83]:
prediction_data

,sub1,sub2,sub3,sub4,sub5,internal_sub1,internal_sub2
id,,,,,,,
3116945,0,0,0,0,0,0,0
3116946,1,1,1,1,1,1,1
3116947,1,1,1,1,1,1,1
3116948,1,1,1,1,1,1,1
3116949,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
5194904,1,1,1,1,1,1,1
5194905,1,1,1,1,1,1,1
5194906,1,1,1,1,1,1,1


The SHAP representation from XGBoost only shows the strongest feature impact with the stem-width while the rest of them captures the strongest feature impact with the gill-attachment. 

Under the hypothesis that XGBoost captures slightly different patterns from the dataset, the weight for XGBoost has been given significantly higher than the rest of the models whose strongest feature impact is gill-attachment.

In [84]:
weights = [0, 0, 0, 0, 0, 6, 0]

for i in range(1, len(prediction_data.columns) - 1):
    prediction_data[f"sub{i}"] = prediction_data[f"sub{i}"] * weights[i - 1]

for i in range(1, 3):
    prediction_data[f"internal_sub{i}"] = prediction_data[f"internal_sub{i}"] * weights[i + 4]

In [85]:
prediction_data

,sub1,sub2,sub3,sub4,sub5,internal_sub1,internal_sub2
id,,,,,,,
3116945,0,0,0,0,0,0,0
3116946,0,0,0,0,0,6,0
3116947,0,0,0,0,0,6,0
3116948,0,0,0,0,0,6,0
3116949,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
5194904,0,0,0,0,0,6,0
5194905,0,0,0,0,0,6,0
5194906,0,0,0,0,0,6,0


In [86]:
from sklearn.preprocessing import MinMaxScaler

output = pd.DataFrame(prediction_data.sum(axis = 1).tolist(), columns = ['class'])
print(output)
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(output)

output = min_max_scaler.transform(output)
output_df = pd.DataFrame(prediction_data.index, columns = ['id'])
output_df['class'] = output
print(output_df.head())

         class
0            0
1            6
2            6
3            6
4            0
...        ...
2077959      6
2077960      6
2077961      6
2077962      0
2077963      0

[2077964 rows x 1 columns]
        id  class
0  3116945    0.0
1  3116946    1.0
2  3116947    1.0
3  3116948    1.0
4  3116949    0.0


In [87]:
output_df['class'] = output_df['class'].apply(lambda x: 'e' if x < 0.5 else 'p')
output_df['class'].unique()

array(['e', 'p'], dtype=object)

In [88]:
len(output_df[output_df['class'] == 'e']) / len(output_df)

0.4546180780802747

In [89]:
from sklearn.preprocessing import StandardScaler

output = pd.DataFrame(prediction_data.sum(axis = 1).tolist(), columns = ['class'])

scaler = StandardScaler()

output_df = pd.DataFrame(prediction_data.index, columns = ['id'])
output_df['class'] = pd.DataFrame(scaler.fit_transform(output), columns = ['class'])
output_df['class'] = output_df['class'].apply(lambda x: 'e' if x < 0 else 'p')
len(output_df[output_df['class'] == 'e']) / len(output_df)

0.4546180780802747

In [90]:
%%time 
output_df.to_csv("submission.csv", index = False)

CPU times: user 2.35 s, sys: 55 ms, total: 2.4 s
Wall time: 2.4 s
